{{title_s1_5}}

So far in this tutorial, we've demonstrated how to read Sentinel-1 RTC imagery from two sources and assemble analysis-ready data cubes with appropriate metadata. Now, we'll perform a comparison of the two datasets. 

::::{tab-set}
:::{tab-item} Dataset comparison

While the two datasets are very similar, there are a few key differences:  
1) They use different sources images.   
    - ASF Sentinel-1 RTC imagery is processed from Single Look Complex ([SLC](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-1-sar/products-algorithms/level-1-algorithms/single-look-complex)) images while Planetary Computer Sentinel-1 RTC imagery is processed from Ground Range Detected ([GRD](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-1-sar/products-algorithms/level-1-algorithms/ground-range-detected)) images. SLC images contain both amplitude and phase information for each pixel. They are in radar coordinates and have not yet been multi-looked. In contrast, GRD images has been detected, multi-looked and projected to ground range.

2) They use different digital elevation models (DEMs) for terrain correction.  
    - ASF uses the [GLO-30 Copernicus DEM](https://dataspace.copernicus.eu/explore-data/data-collections/copernicus-contributing-missions/collections-description/COP-DEM) while Planetary Computer uses a Planet DEM.  
3) The datasets have different pixel spacings. For Planetary Computer, the pixel spacing is 10m in both range and azimuth directions. ASF has the option to produce images with 30 m, 20 m, or 10 m pixel spacing. The data used in this tutorial is 30 m. Note that there are tradeoffs in processing time and file size with pixel spacing, see more discussion [here](https://hyp3-docs.asf.alaska.edu/guides/rtc_product_guide/#pixel-spacing_1).  
4) Each platform uses a different algorithm for RTC processing.
5) The ASF dataset comes with an associated layover shadow map for each scene while the Planetary Computer dataset does not.  

All of the above information and much more detail about the processing methods for both datasets are available in each dataset's documentation pages:
- [ASF Sentinel-1 RTC Product Guide](https://hyp3-docs.asf.alaska.edu/guides/rtc_product_guide/#pixel-spacing_1)  
- [Microsoft Planetary Computer Sentinel-1 RTC dataset](https://planetarycomputer.microsoft.com/dataset/sentinel-1-rtc) 
::: 

:::{tab-item} Outline

(content.Section_A)=
**[A. Read and prepare data](#a-read-and-prepare-data)**  
- {{a1_s1_nb5}}  

(content.Section_B)=
**[B. Ensure direct comparison between datasets](#b-ensure-direct-comparison-between-datasets)**
- {{b1_s1_nb5}}
- {{b2_s1_nb5}}
- {{b3_s1_nb5}}

(content.Section_C)=
**[C. Combine objects](#c-combine-objects)**
- {{c1_s1_nb5}}
- {{c2_s1_nb5}}

(content.Section_D)=
**[D. Visualize comparisons](#d-visualize-comparisons)**
- {{d1_s1_nb5}}
- {{d2_s1_nb5}}

:::
:::{tab-item} Learning goals

{{concepts}}
- Comparing and evaluating multiple datasets
- Organizing data so that its structure matches your use-case

{{techniques}}
- Conditional selection based on non-dimensional coordinates using `xr.Dataset.where()`
- Subsetting datasets based on dimensional coordinates using `xr.DataArray.isin()`
- Adding dimensional and non-dimensional coordinates to `xr.Dataset` objects
- Xarray plotting methods
- Projecting xarray objects to different grids using `xr.interp_like()`
:::
::::


In [ ]:
# %xmode minimal
import hvplot.xarray
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import warnings

import s1_tools

warnings.simplefilter(action="ignore", category=FutureWarning)
%matplotlib inline

{{break}}

## A. Read and prepare data

At the end of notebook 3, we wrote the analysis-ready ASF Sentinel-1 data cube that had been clipped to a smaller spatial area of interest to disk. We'll read that into memory now to use in this comparison.

We used Jupyter cell magic to persist the Planetary Computer data cube created in notebook 4. Now we can read it into our notebook by adding `-r` to the store magic command used to persist it. Read more about `storemagic` [here](https://ipython.readthedocs.io/en/stable/config/extensions/storemagic.html).

In [ ]:
%store -r da
pc_cube = da

In [ ]:
pc_cube = pc_cube.compute()

In [ ]:
timeseries_type = "full"

asf_cube = xr.open_dataset(
    f"../data/raster_data/{timeseries_type}_timeseries/intermediate_cubes/s1_asf_clipped_cube.zarr",
    engine="zarr",
    chunks="auto",
    decode_coords="all",
)

Rename the temporal dimension of the ASF dataset to match that of the PC dataset:

In [ ]:
asf_cube = asf_cube.rename({"acq_date": "time"})

In [ ]:
asf_cube = asf_cube.compute()

### {{a1_s1_nb5}}

First, make sure that both objects are projected to the same CRS.

In [ ]:
assert pc_cube.rio.crs == asf_cube.rio.crs, "CRS of both data cubes are expected to be identical."

Let's also check how missing data is handled in both objects. We want missing data to be assigned NaN values.

In [ ]:
asf_cube["vv"].rio.nodata

In [ ]:
pc_cube.sel(band="vv").rio.nodata

The `pc_cube` array contains nan values, but it doesn't have an encoding specifying what value is used to represent nodata. We can assign a nodata value to the dataset below. See Rioxarray's [Nodata Management documentation](https://corteva.github.io/rioxarray/stable/getting_started/nodata_management.html) for more detail on this.

In [ ]:
pc_cube.rio.write_nodata(np.nan, inplace=True)
pc_cube.rio.nodata

In [ ]:
assert (
    np.isnan(asf_cube.vh.rio.nodata) == np.isnan(pc_cube.sel(band="vh").rio.nodata) == True
), "Expected vh nodata value to be np.nan"
assert (
    np.isnan(asf_cube.vv.rio.nodata) == np.isnan(pc_cube.sel(band="vv").rio.nodata) == True
), "Expected vv nodata value to be np.nan"

## B. Ensure direct comparison between datasets

In notebook 3, we removed time steps from the ASF time series where the area of interest was only partially covered by the satellite footprint. Let's do the same for the PC dataset: 

Thanks to all of the metadata wrangling we did in earlier notebooks, we can quickly access information needed to ensure a direct comparison of time steps between the two datasets

### {{b1_s1_nb5}}

Make a list of the acquisition dates in the ASF time series:

In [ ]:
asf_acq_dates = asf_cube.time.dt.date.data.tolist()

Subset the PC time series to only the time steps that exist in the ASF time series:

In [ ]:
pc_subset = pc_cube.where(pc_cube.time.dt.date.isin(asf_acq_dates), drop=True)

Make sure that the time steps we're excluding only have partial coverage:

In [ ]:
pc_outtakes = (
    pc_cube.where(~pc_cube.time.dt.date.isin(asf_acq_dates), drop=True).to_dataset(dim="band").drop_dims("band")
)

In [ ]:
max_pixels = pc_outtakes.mean(dim="time").x.shape[0] * pc_outtakes.mean(dim="time").y.shape[0]
valid_pixels = pc_outtakes.vv.count(dim=["x", "y"])
pc_outtakes["cov"] = (valid_pixels / max_pixels) * 100

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
pc_outtakes.cov.plot.scatter(x="time", y="cov", ax=ax)
ax.set_ylabel("Coverage (%)")
ax.set_title("None")
fig.suptitle("% Coverage of excluded time steps");
# ax.set_ylim(0,100);

Great, we've successfully removed the time steps with limited coverage. Make sure that the time steps of the two datasets are now identical:

In [ ]:
np.testing.assert_array_equal(asf_cube.time.dt.date.data, pc_subset.time.dt.date.data)

Now that we know we are looking at the same time steps across the two datasets, let's take a look at the data for a few individual scenes side-by-side to get a better image of the differences still remaining between the two data cubes. 


In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(12, 5), layout="constrained")

backscatter_kwargs = {"cmap": "Greys_r", "cbar_kwargs": {"label": "dB"}}
ls_kwargs = {
    "cmap": "tab20b",
    "vmin": 0,
    "vmax": 32,
    "cbar_kwargs": {"label": "layover-shadow map"},
}

s1_tools.power_to_db(pc_subset.isel(time=1).sel(band="vv")).plot(ax=axs[0][0], **backscatter_kwargs)
s1_tools.power_to_db(asf_cube.isel(time=1)).vv.plot(ax=axs[0][1], **backscatter_kwargs)
asf_cube.isel(time=1).ls.plot(ax=axs[0][2], **ls_kwargs)

s1_tools.power_to_db(pc_subset.isel(time=10).sel(band="vv")).plot(ax=axs[1][0], **backscatter_kwargs)
s1_tools.power_to_db(asf_cube.isel(time=10)).vv.plot(ax=axs[1][1], **backscatter_kwargs)
asf_cube.isel(time=10).ls.plot(ax=axs[1][2], **ls_kwargs)

col_names = ["PC backscatter", "ASF backscatter", "ASF layover-shadow map"]

for i in range(axs.shape[0]):
    for j in range(axs.shape[1]):
        if i == 0:
            axs[i, j].set_title(col_names[j])
        else:
            axs[i, j].set_title(None)

        axs[i, j].tick_params(axis="x", labelrotation=45)
        axs[i, j].set_ylabel(None)
        axs[i, j].set_xlabel(None)

fig.supylabel("Y coordinate of projection (m)", x=-0.05)
fig.supxlabel("X coordinate of projection (m)", y=-0.10)
fig.suptitle(
    f"Comparing PC (left), ASF (center) backscatter images at two time steps: \n {asf_cube.isel(time=1).time.dt.date.data.astype(str)} (top) and {asf_cube.isel(time=10).time.dt.date.data.astype(str)} (bottom) \n Right column shows ASF layover-shadow map",
    fontsize=14,
    y=1.15,
);

As discussed at the top of this notebook, the PC dataset has a higher spatial resolution than the ASF dataset. The ASF dataset masks pixels that are in radar shadow on the associated layover-shadow map. The PC dataset does not mask out pixels in the same way the the ASF dataset does. It appears that many pixels near the masked regions in the ASF dataset are very dark, and that they are smaller than the shadow regions in the ASF images, which makes sense given the higher spatial resolution. 

To compare the two backscatter datasets, we need to align them on a common spatial grid and apply the same mask that the ASF dataset has to the PC dataset. 

### {{b2_s1_nb5}}

Different approaches to regrid the dataset exist. Here, we will use `xr.interp_like()`. For more discussion and examples of regridding approaches with Xarray, we recommend the Project Pythia Cookbook, [*(re)Gridding with Xarray*](https://projectpythia.org/gridding-cookbook/README.html).

In [ ]:
asf_da = asf_cube.to_dataarray(dim="band")

In [ ]:
pc_downsample = pc_subset.interp_like(asf_da)

Check that the x and y dimensions of the ASF and PC datasets are the same:

In [ ]:
assert set(asf_da.shape) == set(pc_downsample.shape)

In [ ]:
# We don't use xr.da.equals() because the coords are not the same
np.testing.assert_array_equal(asf_da.x.data, pc_downsample.x.data)
np.testing.assert_array_equal(asf_da.y.data, pc_downsample.y.data)

Take a look at the downsampled object:

In [ ]:
pc_downsample

`xr.interp_like()` interpolates the values from the PC dataset (`pc_subset`) onto the grid of `asf_da`. Notice that, while the resultant object has the expected dimensions, the band dimension is not indexed. To fix this, use [`xr.set_index()`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.set_index.html) to create an index from the `band` coordinate variable.

In [ ]:
pc_downsample = pc_downsample.set_index(band='band')
pc_downsample

Great, now the data are aligned on the same spatial grid with proper indexes, but we still need to mask the data in the PC dataset categorized as 'shadow' in the ASF dataset.

### {{b3_s1_nb5}}

We'll use [`xr.where()`](https://docs.xarray.dev/en/stable/generated/xarray.where.html) to assign NaN to all pixels in the PC dataset where the corresponding pixel in the ASF dataset is NaN:

In [ ]:
pc_mask = xr.where(asf_da.notnull(), pc_downsample, np.nan)

Let's check that the masking operation did what we expect it to do:

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12, 4))

date_target = "2021-12-2"
date_actual = asf_da.sel(time=date_target, method="nearest").time.dt.date.data.astype(str)
fig.suptitle(f"{date_actual}: Comparing backscatter images " + r"$\bf{before}$" + " masking")
s1_tools.power_to_db(asf_da.sel(band="vv").sel(time=date_target, method="nearest")).plot(cmap=plt.cm.Greys_r, ax=ax[0])
ax[0].set_title("ASF")
s1_tools.power_to_db(pc_downsample.sel(band="vv").sel(time=date_target, method="nearest")).plot(
    cmap=plt.cm.Greys_r, ax=ax[1], cbar_kwargs={"label": "backscatter"}
)
ax[1].set_title("PC")

ax[0].set_ylabel(None)
ax[0].set_xlabel(None)

ax[1].set_ylabel(None)
ax[1].set_xlabel(None);

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12, 4))

date_target = "2021-12-2"
date_actual = asf_da.sel(time=date_target, method="nearest").time.dt.date.data.astype(str)
fig.suptitle(f"{date_actual}: Comparing backscatter images " + r"$\bf{after}$" + " masking")
s1_tools.power_to_db(asf_da.sel(band="vv").sel(time=date_target, method="nearest")).plot(cmap=plt.cm.Greys_r, ax=ax[0])
ax[0].set_title("ASF")
s1_tools.power_to_db(pc_mask.sel(band="vv").sel(time=date_target, method="nearest")).plot(
    cmap=plt.cm.Greys_r, ax=ax[1], cbar_kwargs={"label": "dB"}
)
ax[1].set_title("PC")

ax[0].set_ylabel(None)
ax[0].set_xlabel(None)

ax[1].set_ylabel(None)
ax[1].set_xlabel(None);

## C. Combine objects

The two datasets are ready to compare. So far in this book, we've been treating each data cube as an independent unit of analysis containing backscatter data, so it made sense to have dimensions `('x','y','time')` or `('x','y','time','band')`. Now, we're interested in comparing the backscatter values between the two datasets. In effect, this new objective implies a new dimension on the datacube, `'source'`.

### {{c1_s1_nb5}}

In [ ]:
asf_da = asf_da.expand_dims("source")
pc_mask = pc_mask.expand_dims("source")

In [ ]:
asf_da["source"] = ("source", ["asf"])
pc_mask["source"] = ("source", ["pc"])

### {{c2_s1_nb5}}

Now both datasets can be combined into a single data cube along the `'source'` dimension:

In [ ]:
comparison_obj = xr.combine_by_coords([asf_da, pc_mask])
comparison_obj

## D. Visualize comparisons

We're ready to visualize backscatter from both datasets. Because we've made a data cube whose dimensionality reflects the comparison, we can use Xarray's plotting features and visualize the comparisons from a single object.

### {{d1_s1_nb5}}

Look at VV backscatter first:

In [ ]:
# Plot backscatter data
vv_fg = s1_tools.power_to_db(comparison_obj.sel(band="vv").mean(dim="time")).plot(
    col="source", cmap=plt.cm.Greys_r, cbar_kwargs=({"label": "dB"})
)
# Format figure and axes
vv_fg.fig.suptitle("Comparing VV backscatter from ASF and PC datasets")
vv_fg.fig.supxlabel("X coordinate of projection (m)")
vv_fg.fig.supylabel("Y coordinate of projection (m)")
vv_fg.fig.set_figheight(7)
vv_fg.fig.set_figwidth(12)

for i in range(len(vv_fg.axs[0])):
    vv_fg.axs[0][i].set_xlabel(None)
    vv_fg.axs[0][i].set_ylabel(None)
vv_fg.axs[0][0].set_title("ASF")
vv_fg.axs[0][1].set_title("PC");

Then VH:

In [ ]:
# Plot backscatter data
vh_fg = s1_tools.power_to_db(comparison_obj.sel(band="vh").mean(dim="time")).plot(
    col="source", cmap=plt.cm.Greys_r, cbar_kwargs=({"label": "dB"})
)

# Figure and axes formatting
vh_fg.fig.suptitle("Comparing VH backscatter from ASF and PC datasets")
vh_fg.fig.supxlabel("X coordinate of projection (m)")
vh_fg.fig.supylabel("Y coordinate of projection (m)")
vh_fg.fig.set_figheight(7)
vh_fg.fig.set_figwidth(12)
for i in range(len(vh_fg.axs[0])):
    vh_fg.axs[0][i].set_xlabel(None)
    vh_fg.axs[0][i].set_ylabel(None)
vh_fg.axs[0][0].set_title("ASF")
vh_fg.axs[0][1].set_title("PC");

### {{d2_s1_nb5}}

Instead of computing mean backscatter values along the time dimension, reduce along the spatial dimensions (x and y) to see backscatter variability over time:

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(14, 8), layout="constrained")
s1_tools.power_to_db(comparison_obj.sel(source="asf", band="vv").mean(dim=["x", "y"])).plot.scatter(
    x="time", ax=ax[0], label="asf", c="b", alpha=0.75
)
s1_tools.power_to_db(comparison_obj.sel(source="pc", band="vv").mean(dim=["x", "y"])).plot.scatter(
    x="time", ax=ax[0], label="pc", c="r", alpha=0.75
)

s1_tools.power_to_db(comparison_obj.sel(source="asf", band="vh").mean(dim=["x", "y"])).plot.scatter(
    x="time", ax=ax[1], label="asf", c="b", alpha=0.75
)
s1_tools.power_to_db(comparison_obj.sel(source="pc", band="vh").mean(dim=["x", "y"])).plot.scatter(
    x="time", ax=ax[1], label="pc", c="r", alpha=0.75
)
ax[0].legend(loc="lower right", bbox_to_anchor=([1, -0.25, 0, 0]))

for i in range(len(ax)):
    ax[i].set_xlabel(None)
    ax[i].set_ylabel("dB")

ax[0].set_title("VV")
ax[1].set_title("VH")

fig.supxlabel("Time")
# fig.supylabel('dB')
fig.suptitle(
    "Comparing mean VV and VH backscatter over time from PC (red) and ASF (blue) datasets",
    fontsize=14,
    y=1.05,
);

We can also use `hvplot` to make an interactive visualization of this comparison:

In [ ]:
asf_plot = s1_tools.power_to_db(
    comparison_obj.sel(source="asf").to_dataset(dim="band")["vv"].mean(dim=["x", "y"])
).hvplot.scatter(x="time", label="asf")
pc_plot = s1_tools.power_to_db(
    comparison_obj.sel(source="pc").to_dataset(dim="band")["vv"].mean(dim=["x", "y"])
).hvplot.scatter(x="time", label="pc")

asf_plot * pc_plot

{{conclusion}}